In [3]:
#all lightnings

import pandas as pd

df = pd.read_csv("../postprocessed/mediterranean lightnings 2009-2021.csv")

df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')

df['Month'] = df['Date'].apply(lambda x: x.strftime('%Y-%m'))

df = df.drop(df.columns[[0,1]], axis=1)

new_order = ['Month', 'Latitude', 'Longitude', 'Power (J)']
df = df.reindex(columns=new_order)

df['Latituder'] = df['Latitude'].round(1)
df['Longituder'] = df['Longitude'].round(1)

df = df.sort_values(['Month', 'Latitude', 'Longitude'])
df = df.reset_index()
df = df.drop(df.columns[0], axis=1)
df = df.drop(df[(df.Month == '2021-12') | (df.Month == '2022-01')].index)


,Month,Latitude,Longitude,Power (J),Latituder,Longituder
0,2009-12,31.1029,29.3571,3072.06,31.0,29.0
1,2009-12,31.1641,33.3012,35083.02,31.0,33.0
2,2009-12,31.2709,33.1156,980.89,31.0,33.0
3,2009-12,31.2804,32.9514,1001.28,31.0,33.0
4,2009-12,31.4229,33.9174,564.70,31.0,34.0
...,...,...,...,...,...,...
1299939,2020-02,45.0633,14.7613,22.69,45.0,15.0
1299940,2020-02,45.0726,14.8398,13.95,45.0,15.0
1299941,2020-02,45.1035,14.8221,91.91,45.0,15.0
1299942,2020-02,45.1593,13.2721,31759.04,45.0,13.0


In [10]:
#superbolts

import pandas as pd

df = pd.read_csv("../postprocessed/mediterranean superbolts 2009-2021.csv")

df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')

df['Month'] = df['Date'].apply(lambda x: x.strftime('%Y-%m'))

df = df.drop(df.columns[[0,1]], axis=1)

new_order = ['Month', 'Latitude', 'Longitude', 'Power (J)']
df = df.reindex(columns=new_order)

df['Latituder'] = df['Latitude'].round(1)
df['Longituder'] = df['Longitude'].round(1)

df = df.sort_values(['Month', 'Latitude', 'Longitude'])
df = df.reset_index()
df = df.drop(df.columns[0], axis=1)
df = df.drop(df[(df.Month == '2021-12') | (df.Month == '2022-01')].index)
df


,Month,Latitude,Longitude,Power (J),Latituder,Longituder
0,2010-01,41.3376,19.1571,1092108.67,41.3,19.2
1,2010-01,41.9583,8.0269,1830018.90,42.0,8.0
2,2011-01,36.0043,35.1179,4126677.12,36.0,35.1
3,2011-01,36.1162,35.8720,2523415.40,36.1,35.9
4,2011-12,35.0026,24.0483,1144003.06,35.0,24.0
...,...,...,...,...,...,...
1499,2020-02,34.1589,35.0078,1132875.43,34.2,35.0
1500,2020-02,37.1402,26.5892,1479165.96,37.1,26.6
1501,2020-02,37.4591,24.1738,1432100.25,37.5,24.2
1502,2020-02,37.7067,24.8482,17667017.97,37.7,24.8


In [ ]:
import pandas as pd 
from math import sqrt

def haversine(lat1, lon1, lat2, lon2):
    latdist = abs(lat1-lat2)
    londist = abs(lon1-lon2)
    diagdist = sqrt(latdist**2 + londist**2 )
    return diagdist

salinity_df = pd.read_csv("../postprocessed/Salinity 2009-2021.csv")

salinity_df = salinity_df[salinity_df['Longitude'] >= -5]

salinity_df['Latituder'] = salinity_df['Latitude'].round(1)
salinity_df['Longituder'] = salinity_df['Longitude'].round(1)

merged_df = pd.merge(df, salinity_df, on=["Month", "Latituder", "Longituder"], how='left')

merged_df['distance'] = merged_df.apply(lambda row: haversine(row['Latitude_x'], row['Longitude_x'], row['Latitude_y'], row['Longitude_y']), axis=1)
merged_df = merged_df.dropna()

merged_df = merged_df.groupby(['Month', 'Latitude_x', 'Longitude_x']).apply(lambda x: x.loc[x['distance'].idxmin()]).reset_index(drop=True)

merged_df = merged_df.drop(columns=['Latituder', 'Longituder','distance'])
merged_df = merged_df.rename(columns={"Latitude_x":"Latitude","Longitude_x":"Longitude"})

In [3]:
temperature_df = pd.read_csv("../postprocessed/Temperature 2009-2021.csv")

temperature_df = temperature_df.rename(columns={"Latitude":"Latitude_y","Longitude":"Longitude_y"})

merged_df = pd.merge(merged_df, temperature_df, on=["Month", "Latitude_y", "Longitude_y"], how='left')

In [4]:
acidity_df = pd.read_csv("../postprocessed/Acidity 2009-2021.csv")

acidity_df = acidity_df.rename(columns={"Latitude":"Latitude_y","Longitude":"Longitude_y"})

merged_df = pd.merge(merged_df, acidity_df, on=["Month", "Latitude_y", "Longitude_y"], how='left')

In [5]:
carbon_df = pd.read_csv("../postprocessed/Carbon 2009-2021.csv")

carbon_df = carbon_df.rename(columns={"Latitude":"Latitude_y","Longitude":"Longitude_y"})

merged_df = pd.merge(merged_df, carbon_df, on=["Month", "Latitude_y", "Longitude_y"], how='left')

In [6]:
alkalinity_df = pd.read_csv("../postprocessed/Alkalinity 2009-2021.csv")

alkalinity_df = alkalinity_df.rename(columns={"Latitude":"Latitude_y","Longitude":"Longitude_y"})

merged_df = pd.merge(merged_df, alkalinity_df, on=["Month", "Latitude_y", "Longitude_y"], how='left')

In [7]:
merged_df = merged_df.drop(columns=['Latitude_y', 'Longitude_y'])
merged_df

,Month,Latitude,Longitude,Power (J),Sodium,Temperature,pH,Carbon,Alkalinity
0,2010-01,41.3376,19.1571,1092108.67,38.386047,14.993838,8.155515,2.361949,2.703533
1,2010-01,41.9583,8.0269,1830018.90,38.093773,13.631414,8.127534,2.341064,2.643632
2,2011-01,36.0043,35.1179,4126677.12,39.264580,20.127167,8.098685,2.380318,2.748282
3,2011-01,36.1162,35.8720,2523415.40,38.706863,20.015251,8.102674,2.381072,2.746286
4,2011-12,35.0026,24.0483,1144003.06,38.963997,18.164747,8.126753,2.404533,2.770763
...,...,...,...,...,...,...,...,...,...
1357,2020-02,34.1589,35.0078,1132875.43,39.095287,17.714447,8.110338,2.389785,2.737616
1358,2020-02,37.1402,26.5892,1479165.96,39.232460,17.358300,8.118232,2.396134,2.747818
1359,2020-02,37.4591,24.1738,1432100.25,39.107113,15.689024,8.142412,2.412844,2.764086
1360,2020-02,37.7067,24.8482,17667017.97,39.220270,15.819401,8.137862,2.405581,2.754944


In [8]:
merged_df.to_csv("../postprocessed/Biogeochemistry superbolts vector.csv")

In [ ]:
#new

In [5]:
#all lightnings

import pandas as pd

df = pd.read_csv("../postprocessed/mediterranean lightnings 2009-2021.csv")

df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')

df['Month'] = df['Date'].apply(lambda x: x.strftime('%Y-%m'))

df['Latituder'] = df['Latitude'].round(1)
df['Longituder'] = df['Longitude'].round(1)

df = df.drop(df[(df.Month == '2021-12') | (df.Month == '2022-01')].index)

df = df.drop(columns=["Month"],axis=1)

df = df.rename(columns={"Date":"time"})

df


,Date,Latitude,Longitude,Power (J),Latituder,Longituder
0,2009-12-01,37.6420,4.9377,60484.92,37.6,4.9
1,2009-12-01,38.6991,8.3091,3046.67,38.7,8.3
2,2009-12-01,36.9608,4.2523,7787.19,37.0,4.3
3,2009-12-01,39.0087,5.9722,851.00,39.0,6.0
4,2009-12-01,37.3525,7.2369,5440.39,37.4,7.2
...,...,...,...,...,...,...
1299939,2020-02-29,33.3339,34.9043,1461.92,33.3,34.9
1299940,2020-02-29,34.6460,34.8719,29232.37,34.6,34.9
1299941,2020-02-29,34.1010,35.3089,911.27,34.1,35.3
1299942,2020-02-29,33.7268,35.3901,2490.43,33.7,35.4


In [7]:
df2 = pd.read_csv("../daily biogeochemistry.csv")
df2["time"] = df2["time"].str[:-9]
df2['time'] = pd.to_datetime(df2['time'], format='%Y/%m/%d')
df2

,Unnamed: 0,time,latitude,longitude,so,temp,ph,talk,dissic
0,0,2009-12-01 12:00:00,30.312500,18.916666,38.954740,19.959260,8.113097,2.727925,2.356033
1,1,2009-12-01 12:00:00,30.312500,18.958334,38.926605,20.264929,8.109640,2.727126,2.355098
2,2,2009-12-01 12:00:00,30.312500,19.000000,38.916557,20.474968,8.107011,2.726541,2.354309
3,3,2009-12-01 12:00:00,30.312500,19.041666,38.914143,20.547367,8.106014,2.726602,2.354140
4,4,2009-12-01 12:00:00,30.312500,19.083334,38.917503,20.549835,8.106150,2.727115,2.354367
...,...,...,...,...,...,...,...,...,...
143373775,143373775,2020-02-28 12:00:00,45.687500,13.708333,37.262768,10.540786,8.178234,2.749946,2.441770
143373776,143373776,2020-02-28 12:00:00,45.729168,13.541667,33.371845,10.959924,8.157243,2.804251,2.528241
143373777,143373777,2020-02-28 12:00:00,45.729168,13.583333,34.007910,10.739080,8.160475,2.800785,2.520210
143373778,143373778,2020-02-28 12:00:00,45.729168,13.625000,34.865173,10.532424,8.162574,2.794025,2.507789


In [22]:
import pandas as pd 
from math import sqrt

def haversine(lat1, lon1, lat2, lon2):
    latdist = abs(lat1-lat2)
    londist = abs(lon1-lon2)
    diagdist = sqrt(latdist**2 + londist**2 )
    return diagdist


df2['Latituder'] = df2['latitude'].round(1)
df2['Longituder'] = df2['longitude'].round(1)

merged_df = pd.merge(df, df2, on=["time", "Latituder", "Longituder"], how='left')

merged_df['distance'] = merged_df.apply(lambda row: haversine(row['Latitude'], row['Longitude'], row['latitude'], row['longitude']), axis=1)
merged_df = merged_df.dropna()

merged_df = merged_df.groupby(['time', 'Latitude', 'Longitude']).apply(lambda x: x.loc[x['distance'].idxmin()]).reset_index(drop=True)

#merged_df = merged_df.drop(columns=['Latituder', 'Longituder','distance'])
#merged_df = merged_df.rename(columns={"Latitude_x":"Latitude","Longitude_x":"Longitude"})

In [23]:
merged_df

,time,Latitude,Longitude,Power (J),Latituder,Longituder,Unnamed: 0,latitude,longitude,so,temp,ph,talk,dissic,distance
0,2009-12-01,32.2717,16.4187,7325.13,32.3,16.4,9076.0,32.270832,16.416666,38.408913,22.034770,8.072050,2.691513,2.334504,0.002211
1,2009-12-01,32.4561,16.2385,13777.83,32.5,16.2,11100.0,32.479168,16.250000,38.420956,21.909150,8.070853,2.692296,2.337485,0.025776
2,2009-12-01,32.7783,17.4759,4889.98,32.8,17.5,14205.0,32.770832,17.458334,38.160995,20.586388,8.083223,2.671848,2.326656,0.019088
3,2009-12-01,32.7865,15.1156,15367.76,32.8,15.1,14149.0,32.770832,15.125000,38.308590,21.427584,8.075890,2.685109,2.333145,0.018271
4,2009-12-01,32.8098,15.8033,10615.32,32.8,15.8,14650.0,32.812500,15.791667,38.389010,21.648327,8.073261,2.689546,2.336258,0.011942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245092,2020-02-28,41.7588,19.4079,4043.11,41.8,19.4,143359750.0,41.770832,19.416666,36.898075,13.609066,8.157640,2.740279,2.418223,0.014887
1245093,2020-02-28,41.7775,19.3731,1104.86,41.8,19.4,143359749.0,41.770832,19.375000,36.845978,13.702010,8.158886,2.740308,2.417395,0.006933
1245094,2020-02-28,41.8055,19.1850,72.09,41.8,19.2,143360029.0,41.812500,19.166666,37.737923,14.063701,8.165195,2.738853,2.402841,0.019625
1245095,2020-02-28,41.8194,19.1869,35.00,41.8,19.2,143360029.0,41.812500,19.166666,37.737923,14.063701,8.165195,2.738853,2.402841,0.021378


In [25]:
merged_df = merged_df.drop(columns=['Latituder', 'Longituder','distance','Unnamed: 0','latitude','longitude'])
merged_df = merged_df.rename(columns={"so":"Salinity","temp":"Temperature","ph":"Acidity","talk":"Alkalinity","dissic":"Carbon"})
merged_df

,time,Latitude,Longitude,Power (J),Salinity,Temperature,Acidity,Alkalinity,Carbon
0,2009-12-01,32.2717,16.4187,7325.13,38.408913,22.034770,8.072050,2.691513,2.334504
1,2009-12-01,32.4561,16.2385,13777.83,38.420956,21.909150,8.070853,2.692296,2.337485
2,2009-12-01,32.7783,17.4759,4889.98,38.160995,20.586388,8.083223,2.671848,2.326656
3,2009-12-01,32.7865,15.1156,15367.76,38.308590,21.427584,8.075890,2.685109,2.333145
4,2009-12-01,32.8098,15.8033,10615.32,38.389010,21.648327,8.073261,2.689546,2.336258
...,...,...,...,...,...,...,...,...,...
1245092,2020-02-28,41.7588,19.4079,4043.11,36.898075,13.609066,8.157640,2.740279,2.418223
1245093,2020-02-28,41.7775,19.3731,1104.86,36.845978,13.702010,8.158886,2.740308,2.417395
1245094,2020-02-28,41.8055,19.1850,72.09,37.737923,14.063701,8.165195,2.738853,2.402841
1245095,2020-02-28,41.8194,19.1869,35.00,37.737923,14.063701,8.165195,2.738853,2.402841


In [27]:
merged_df.to_csv("NEW VECTOR.csv",index=False)